# Variational Quantum Circuits – VQE ansatz exploration across backends

In [ ]:
import numpy as np
from qiskit import QuantumCircuit

from ariadne import explain_routing, simulate


# Create a VQE-style ansatz circuit
def create_vqe_ansatz(n_qubits=6, depth=3):
    """Create a hardware-efficient VQE ansatz."""
    qc = QuantumCircuit(n_qubits, n_qubits)

    # Initial state preparation
    for i in range(n_qubits):
        qc.h(i)

    # Variational layers
    for _layer in range(depth):
        # Entangling layer (nearest-neighbor)
        for i in range(0, n_qubits - 1, 2):
            qc.cx(i, i + 1)

        # Single-qubit rotations (parameterized)
        for i in range(n_qubits):
            qc.ry(np.random.uniform(0, 2 * np.pi), i)
            qc.rz(np.random.uniform(0, 2 * np.pi), i)

        # Second entangling layer (next-nearest-neighbor)
        for i in range(1, n_qubits - 1, 2):
            qc.cx(i, i + 1)

    qc.measure_all()
    return qc


# Create the ansatz
vqe_circuit = create_vqe_ansatz(6, depth=3)
print(f"VQE ansatz: {vqe_circuit.num_qubits} qubits, {vqe_circuit.depth()} depth")
print(f"Gate counts: {vqe_circuit.count_ops()}")

In [ ]:
# Test automatic routing for the VQE circuit
print("Automatic routing analysis:")
explanation = explain_routing(vqe_circuit)
print(explanation)

# Run with automatic backend selection
result_auto = simulate(vqe_circuit, shots=1000)
print(f"\nAuto-selected backend: {result_auto.backend_used.value}")
print(f"Execution time: {result_auto.execution_time:.4f}s")
print(f"Sample counts: {dict(list(result_auto.counts.items())[:3])}")

In [ ]:
# Compare with explicit backend choices
backends_to_test = ["qiskit", "mps", "tensor_network"]
results = {}

print("\nBackend comparison for VQE circuit:")
print("=" * 50)

for backend in backends_to_test:
    try:
        result = simulate(vqe_circuit, shots=1000, backend=backend)
        results[backend] = result

        # Calculate some basic metrics
        total_shots = sum(result.counts.values())
        unique_outcomes = len(result.counts)

        print(
            f"{backend:15} | {result.execution_time:8.4f}s | {unique_outcomes:4} outcomes | {result.backend_used.value}"
        )

    except Exception as e:
        print(f"{backend:15} | {'FAILED':>8} | {str(e)[:30]}")

In [ ]:
# Analyze result consistency
print("\nResult consistency check:")
print("=" * 30)

# Get the most common outcome from each backend
for backend, result in results.items():
    most_common = max(result.counts.items(), key=lambda x: x[1])
    probability = most_common[1] / sum(result.counts.values())
    print(f"{backend:15}: {most_common[0]} ({probability:.3f})")

This notebook demonstrates how variational circuits like VQE ansätze behave differently across backends. The key insights:

1. **Automatic routing** selects the most appropriate backend based on circuit structure
2. **Performance varies** significantly between backends for the same circuit
3. **Results are consistent** across all working backends (within statistical error)
4. **Backend choice matters** for optimization and research workflows